# Museums in the Pandemic - Extract indicators

**Authors**: Andrea Ballatore (KCL)

**Abstract**: Extract indicators from museum text.

## Setup
This is to check that your environment is set up correctly (it should print 'env ok', ignore warnings).

In [2]:
# Test geospatial libraries
# check environment
import os
print("Conda env:", os.environ['CONDA_DEFAULT_ENV'])
if os.environ['CONDA_DEFAULT_ENV'] != 'mip_v1':
    raise Exception("Set the environment 'mip_v1' on Anaconda. Current environment: " + os.environ['CONDA_DEFAULT_ENV'])

# spatial libraries 
import pandas as pd
import pickle
import spacy
from termcolor import colored
import sys
import numpy as np
from numpy import arange
#import tensorflow as tf
from bs4 import BeautifulSoup
from bs4.element import Comment
#import torch
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

# import from `mip` project
print(os.getcwd())
fpath = os.path.abspath('../')
if not fpath in sys.path:
    sys.path.insert(0, fpath)

out_folder = '../../'

from museums import *
from utils import _is_number
from analytics.text_models import derive_new_attributes_matches, get_all_matches_from_db, get_indicator_annotations

print('env ok')

Conda env: mip_v1
/Users/andreaballatore/Dropbox/DRBX_Docs/Work/Projects/github_projects/museums-in-the-pandemic/mip/notebooks
env ok


## Connect to DB

It needs the DCS VPN active to work.

In [3]:
# open connection to DB
from db.db import connect_to_postgresql_db

db_conn = connect_to_postgresql_db()
print("DB connected")

DB connected


## Extract matches for all museums

Using the best deep learning model defined above, find indicators for all museums (from websites and social media).

### Load deep learning validation model

In [4]:
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler

def remove_duplicate_matches(df):
    # find duplicates
    n = len(df)
    df = df.drop_duplicates(subset=df.columns.difference(['page_id','sentence_id']))
    print('remove_duplicate_matches:',n,len(df))
    return df

def prep_match_data(df):
    for c in valid_model_columns:
        if not c in df.columns:
            print("Warning: column '{}' is missing, adding a zero column".format(c))
            df[c] = 0
    
    df = remove_duplicate_matches(df)
    df = df[valid_model_columns]
    assert len(df.columns) == 33, len(df.columns)
    num_df = df.select_dtypes(include=[np.number])
    scaler = MinMaxScaler()
    # fit and transform in one step
    cols = num_df.columns
    x_data = pd.DataFrame(scaler.fit_transform(num_df),columns=cols)
    return x_data

def convert_pred_to_bool(vals):
    pred_y = (vals > 0.5).astype("bool")
    # unpack results
    bool_vals = [item for sublist in pred_y for item in sublist]
    return bool_vals

In [5]:
# MODEL COLUMNS
cols_fn = out_folder+"data/analysis/matching_validation/matching_validation_deep_learning_model_columns.csv"
valid_model_columns = pd.read_csv(cols_fn).iloc[:, 0].tolist()

valid_ann_df_fn = 'matches_valid_ann_df_v3.pik'
valid_ann_df = pd.read_pickle(out_folder+'data/annotations/'+valid_ann_df_fn)

valid_match_cnn_model = load_model(out_folder+"data/analysis/matching_validation/matching_validation_deep_learning_model.h5")
valid_match_cnn_model

x_data = prep_match_data(valid_ann_df)
assert len(x_data.columns) == 33, len(x_data.columns)
print(x_data)
pred_valid = convert_pred_to_bool(valid_match_cnn_model.predict(x_data))

valid_ann_df['predicted_valid'] = pred_valid

#valid_ann_df.to_excel(out_folder+"tmp/check_deeplearning.xlsx",index=False)
valid_ann_df.sample(10)

2021-11-03 13:08:58.415996: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


remove_duplicate_matches: 700 700
     sem_similarity   token_n   lemma_n  ann_overlap_lemma  ann_overlap_token  \
0          0.659748  0.285714  0.166667           1.000000            1.00000   
1          0.703680  0.428571  0.333333           0.733333            0.75000   
2          0.759582  0.000000  0.000000           0.146667            0.00000   
3          0.726219  0.571429  0.500000           0.644448            0.66667   
4          0.667261  0.000000  0.000000           0.200000            0.00000   
..              ...       ...       ...                ...                ...   
695        0.830129  0.428571  0.333333           0.573333            0.60000   
696        0.712849  0.142857  0.166667           1.000000            0.50000   
697        0.398828  0.000000  0.000000           0.200000            0.00000   
698        0.647905  0.000000  0.000000           0.200000            0.00000   
699        0.661403  0.142857  0.000000           0.288885            0.333

2021-11-03 13:08:58.835392: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


,muse_id,page_id,sentence_id,example_id,indicator_code,session_id,ann_ex_tokens,page_tokens,sem_similarity,token_n,...,indicator_code_project_postpone,indicator_code_reopen_intent,indicator_code_reopen_plan,indicator_code_staff_hiring,indicator_code_staff_restruct,indicator_code_staff_working,overlap_bin,valid_match,valid_match_b,predicted_valid
583,mm.ace.1291,568767,mus_page568767_sent00005,ann_ex_00246,reopen_intent,20210304,look forward welcoming you again soon,what can you learn our parish,0.7951,1,...,0,1,0,0,0,0,"(0.0, 0.45]",F,False,False
617,mm.domus.SE556,411406,mus_page411406_sent00089,ann_ex_00017,closed_cur,20210304,our office wil not reopen,however where space available weekdays pre boo...,0.7786,1,...,0,0,0,0,0,0,"(0.0, 0.45]",F,False,False
312,mm.domus.SE379,245402,mus_page245402_sent00019,ann_ex_00185,online_event,20210304,online event,online exhibitions stay connected follow us so...,0.7167,1,...,0,0,0,0,0,0,"(0.45, 1.01]",T,True,False
241,mm.mald.124,57721,mus_page57721_sent00020,ann_ex_00036,closed_cur,20210304,line government restrictions remain closed now,safety our staff volunteers visitors local com...,0.8125,2,...,0,0,0,0,0,0,"(0.45, 1.01]",T,True,True
254,mm.aim82NM.012,591936,mus_page591936_sent00042,ann_ex_00242,reopen_intent,20210304,we look forward welcoming back soon able safe,rescue finished he took oilskins went back rec...,0.7729,1,...,0,1,0,0,0,0,"(0.0, 0.45]",F,False,False
44,mm.mgs.348,83564,mus_page83564_sent00050,ann_ex_00172,online_engag,20210304,new blog,more britannia blog read our blog to learn lif...,0.7909,1,...,0,0,0,0,0,0,"(0.45, 1.01]",T,True,True
675,mm.musa.118,708221,mus_page708221_sent00006,ann_ex_00180,online_event,20210304,online events,blog events shop contact,0.7712,1,...,0,0,0,0,0,0,"(0.45, 1.01]",F,False,False
699,mm.wiki.398,65086,mus_page65086_sent00023,ann_ex_00195,open_cafe,20210304,our cafe open,key facts summary cafe learning news contact p...,0.7252,1,...,0,0,0,0,0,0,"(0.0, 0.45]",F,False,False
81,mm.domus.SC034,941439,mus_page941439_sent00009,ann_ex_00229,reopen_intent,20210304,when we open season,we remain open appointment researchers special...,0.7164,2,...,0,1,0,0,0,0,"(0.45, 1.01]",T,True,True
77,mm.domus.SE480,636198,mus_page636198_sent00017,ann_ex_00036,closed_cur,20210304,line government restrictions remain closed now,result current situation covid-19 pandemic gov...,0.8802,3,...,0,0,0,0,0,0,"(0.45, 1.01]",T,True,True


In [65]:
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_fscore_support, accuracy_score

def aggr_indicators_by_indic(df):
    d = {}
    for c in ['indicator_code']:
        d[c] = df[c].tolist()[0]
    d['cases_700'] = len(df)
    d['precision'], d['recall'], d['fscore'], support = precision_recall_fscore_support(df.valid_match_b, df.predicted_valid, average='binary')
    confmat = confusion_matrix(df.valid_match_b, df.predicted_valid, normalize='all')
    if len(confmat)>1:
        d['tn'], d['fp'], d['fn'], d['tp'] = confmat.ravel()
    for k,v in d.items():
        d[k] = [v]
        
    res = pd.DataFrame.from_dict(d)
    return res

# extract indicator stats
#indf2 = valid_ann_df.indicator_code.value_counts().to_frame('valid_set_700_cases')
#indf2['indicator_code'] = indf2.index
#indf2


indf2 = pd.DataFrame()
for indic,subdf in valid_ann_df.groupby('indicator_code'):
    indf2 = indf2.append(aggr_indicators_by_indic(subdf))
indf2 = indf2.round(3)

/Users/andreaballatore/opt/anaconda3/envs/mip_v1/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/andreaballatore/opt/anaconda3/envs/mip_v1/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/andreaballatore/opt/anaconda3/envs/mip_v1/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start

### Load all matches from DB

- Dump all matches from DB, after running `an_text` on ALL museums for a given crawling session.

In [31]:
# DB columns:
""" 
example_id indicator_code lemma_n lemma_n_wdupl token_n token_n_wdupl criticalwords_n criticalwords_n_wdupl sentence_id  sent_len
example_len example_crit_len ann_overlap_lemma ann_overlap_token ann_overlap_criticwords txt_overlap_lemma
txt_overlap_token ann_ex_tokens ann_ex_tokens page_tokens session_id page_id muse_id keep_stopwords
"""

# load from DB - SLOW
sessions = ['20210304','20210404','20210914']
for session_id in sessions:
    get_all_matches_from_db(session_id, db_conn, out_folder)

get_all_matches_from_db 20210304
query results: (1136440, 17)
	saved ../../tmp/matches_dump_df_20210304.pik
get_all_matches_from_db 20210404
query results: (842167, 17)
	saved ../../tmp/matches_dump_df_20210404.pik


### Predict all matches

In [55]:
sessions = ['20210304']#,'20210404']

def select_valid_matches(df, model):
    """ use Deep Learning model to validate matches """
    x_data = prep_match_data(df)
    
    print('select_valid_matches', x_data.shape)
    # check column order
    assert valid_model_columns == x_data.columns.tolist()
    print(x_data.shape)
    # apply model for predictions
    valid_int = model.predict(x_data)
    pred_valid = convert_pred_to_bool(valid_int)
    #print(type(pred_valid),len(pred_valid))
    df['valid_match'] = pred_valid
    print(df.valid_match.value_counts())
    return df

assert len(valid_model_columns) > 0
allsess_match_df = pd.DataFrame(columns=valid_model_columns)

for session_id in sessions:
    print('> session_id',session_id)
    matches_fn = out_folder+'tmp/matches_dump_df_{}.pik'.format(session_id)
    matchdf = pd.read_pickle(matches_fn)
    matchdf = remove_duplicate_matches(matchdf)
    print("\t", matches_fn, matchdf.shape)
    # apply model to get valid matches
    validmatch_df = select_valid_matches(matchdf, valid_match_cnn_model)
    # save sample to inspect results
    validmatch_df.sample(200).to_csv(out_folder+'tmp/valid_matches_sample_{}.tsv'.format(session_id),sep='\t')
    # save results
    allsess_match_df = pd.concat([allsess_match_df, validmatch_df])

print('all matches:',len(allsess_match_df))

> session_id 20210304
remove_duplicate_matches: 1136440 926928
	 ../../tmp/matches_dump_df_20210304.pik (926928, 41)
remove_duplicate_matches: 926928 926928
select_valid_matches (926928, 33)
(926928, 33)
False    842258
True      84670
Name: valid_match, dtype: int64
all matches: 926928


In [131]:
print(allsess_match_df.columns)
print("Matches from DB:")
round(allsess_match_df['valid_match'].value_counts()/len(allsess_match_df),2)

Index(['sem_similarity', 'token_n', 'lemma_n', 'ann_overlap_lemma',
       'ann_overlap_token', 'example_len', 'txt_overlap_lemma',
       'txt_overlap_token', 'ann_overlap_criticwords', 'lemmatoken_n',
       'ann_overlap_tokenlemma', 'txt_overlap_tokenlemma',
       'indicator_code_closed_indef', 'indicator_code_closed_perm',
       'indicator_code_finance_health', 'indicator_code_funding_did_not_get',
       'indicator_code_funding_fundraise', 'indicator_code_funding_gov_emer',
       'indicator_code_funding_other_emer', 'indicator_code_lang_difficulty',
       'indicator_code_made_covid_safe', 'indicator_code_online_engag',
       'indicator_code_online_event', 'indicator_code_online_exhib',
       'indicator_code_open_cafe', 'indicator_code_open_cur',
       'indicator_code_open_onlineshop', 'indicator_code_project_postpone',
       'indicator_code_reopen_intent', 'indicator_code_reopen_plan',
       'indicator_code_staff_hiring', 'indicator_code_staff_restruct',
       'indicator

False    0.91
True     0.09
Name: valid_match, dtype: float64

## Aggregate indicators

In [68]:
assert len(allsess_match_valid_df) > 0
allsess_match_valid_df = allsess_match_df[allsess_match_df.valid_match]
allsess_match_valid_df = remove_duplicate_matches(allsess_match_valid_df)
print("N =", len(allsess_match_valid_df))
print("N museums =", len(allsess_match_valid_df.muse_id.unique()))
allsess_match_valid_df.head(100)

NameError: name 'allsess_match_valid_df' is not defined

In [69]:
# load annotations
indic_df, ann_df = get_indicator_annotations(out_folder)
del indic_df
ann_stats_df = ann_df.groupby(['indicator_code']).size().reset_index(name='n_indic_all_ann_examples')
print(ann_df.head(30))
ann_stats_df

      example_id                                       text_phrases  \
1   ann_ex_00002  closed to members of the public until further ...   
2   ann_ex_00003                       closed until further notice    
5   ann_ex_00006  currently we are closed due to Covid restricti...   
6   ann_ex_00007                                 had to close doors   
10  ann_ex_00011  there will be no services over due to the Covi...   
15  ann_ex_00016  currently closed due to Government Covid restr...   
16  ann_ex_00017                     our office wil not reopen till   
17  ann_ex_00018                         is now closed due to covid   
19  ann_ex_00020  We have made the decision to remain closed to ...   
21  ann_ex_00022  we have taken the hard decision to remain clos...   
22  ann_ex_00023   we have taken the hard decision to remain closed   
23  ann_ex_00024        will be closed during the national lockdown   
24  ann_ex_00025  currently closed in line with Government restr...   
25  an

,indicator_code,n_indic_all_ann_examples
0,closed_cur,21
1,closed_indef,4
2,closed_perm,4
3,finance_health,6
4,funding_did_not_get,7
5,funding_fundraise,45
6,funding_gov_emer,11
7,funding_other_emer,4
8,lang_difficulty,37
9,made_covid_safe,1


In [71]:
# get indicator stats

def aggr_indicators_by_indic_allmatches(df):
    d = {}
    for c in ['indicator_code']:
        d[c] = df[c].tolist()[0]
    d['matches_ml_true'] = len(df)
    d['matches_ml_tot'] = len(allsess_match_df)
    d['matches_ml_pc'] = round(d['matches_ml_true'] / d['matches_ml_tot'],3)
    #d['precision'], d['recall'], d['fscore'], support = precision_recall_fscore_support(df.valid_match_b, df.predicted_valid, average='binary')
    #confmat = confusion_matrix(df.valid_match_b, df.predicted_valid, normalize='all')
    #if len(confmat)>1:
    #    d['tn'], d['fp'], d['fn'], d['tp'] = confmat.ravel()
    for k,v in d.items():
        d[k] = [v]
        
    res = pd.DataFrame.from_dict(d)
    return res

indf3 = pd.DataFrame()
for indic,subdf in allsess_match_df.groupby('indicator_code'):
    indf3 = indf3.append(aggr_indicators_by_indic_allmatches(subdf))

indf3 = indf3.reset_index()
indf3 = indf3.merge(indf2,on='indicator_code',how='outer')
indf3 = indf3.merge(ann_stats_df,on='indicator_code',how='outer')
indf3.to_excel(out_folder+'tmp/indicator_code_stats_validation.xlsx',index=False)
indf3

,index,indicator_code,matches_ml_true,matches_ml_tot,matches_ml_pc,cases_700,precision,recall,fscore,tn,fp,fn,tp,n_indic_all_ann_examples
0,0,closed_cur,121018,926928,0.131,83.0,0.886,0.705,0.785,0.422,0.048,0.157,0.373,21
1,0,closed_indef,37087,926928,0.040,22.0,0.889,0.889,0.889,0.545,0.045,0.045,0.364,4
2,0,closed_perm,11632,926928,0.013,4.0,0.000,0.000,0.000,NaN,NaN,NaN,NaN,4
3,0,finance_health,8005,926928,0.009,7.0,1.000,1.000,1.000,0.714,0.000,0.000,0.286,6
4,0,funding_did_not_get,27046,926928,0.029,21.0,0.000,0.000,0.000,0.952,0.000,0.048,0.000,7
5,0,funding_fundraise,128397,926928,0.139,74.0,0.909,0.357,0.513,0.608,0.014,0.243,0.135,45
6,0,funding_gov_emer,50763,926928,0.055,19.0,1.000,0.500,0.667,0.895,0.000,0.053,0.053,11
7,0,funding_other_emer,8167,926928,0.009,2.0,0.000,0.000,0.000,NaN,NaN,NaN,NaN,4
8,0,lang_difficulty,72410,926928,0.078,67.0,1.000,0.125,0.222,0.881,0.000,0.104,0.015,37
9,0,made_covid_safe,11831,926928,0.013,5.0,0.000,0.000,0.000,0.800,0.000,0.200,0.000,1


In [187]:
col_aggr = ['muse_id','session_id','page_id','indicator_code','n_indic_all_ann_examples']

def aggr_indicators_sent(df):
    d = {}
    for c in col_aggr:
        d[c] = df[c].tolist()[0]
    d['n_uniq_sentences'] = df['sentence_id'].nunique()
    d['n_matched_annotations'] = df['example_id'].nunique()
    d['n_matches'] = len(df)
    d['matches_to_sent_ratio'] = round(d['n_matches'] / d['n_uniq_sentences'],3)
    d['matches_to_example_ratio'] = round(d['n_matches'] / d['n_indic_all_ann_examples'],3)
    #d['matches_ratio'] = round(d['n_matches'] / d['n_indic_all_ann_examples'],3)
    return pd.Series(d)

n = len(allsess_match_valid_df)
allsess_match_valid_df2 = allsess_match_valid_df.merge(ann_stats_df, on='indicator_code')
assert n == len(allsess_match_valid_df2)

muse_indic_sent_df = allsess_match_valid_df2.groupby(col_aggr).apply(aggr_indicators_sent)
print(muse_indic_sent_df.columns)
muse_indic_sent_df.reset_index(drop=True, inplace=True)
muse_indic_sent_df = muse_indic_sent_df.sort_values(['session_id','muse_id','indicator_code'])
#print(muse_indic_sent_df.session_id.value_counts())
muse_indic_sent_df.to_excel(out_folder+'tmp/museum_indicators_sent_stats-v1.xlsx', index=False)
muse_indic_sent_df.head(30)

Index(['muse_id', 'session_id', 'page_id', 'indicator_code',
       'n_indic_all_ann_examples', 'n_uniq_sentences', 'n_matched_annotations',
       'n_matches', 'matches_to_sent_ratio', 'matches_to_example_ratio'],
      dtype='object')


,muse_id,session_id,page_id,indicator_code,n_indic_all_ann_examples,n_uniq_sentences,n_matched_annotations,n_matches,matches_to_sent_ratio,matches_to_example_ratio
0,domus.NE043,20210304,643977.0,closed_cur,21,5,1,5,1.000,0.238
1,domus.NE043,20210304,643977.0,closed_indef,4,2,1,2,1.000,0.500
2,domus.NE043,20210304,643977.0,online_engag,12,2,2,2,1.000,0.167
3,domus.NE043,20210304,643977.0,open_cur,3,1,1,1,1.000,0.333
4,domus.NE043,20210304,643977.0,project_postpone,3,1,1,1,1.000,0.333
5,domus.NE043,20210304,643977.0,reopen_intent,35,4,1,4,1.000,0.114
6,mm.MDN.006,20210304,706877.0,closed_cur,21,7,8,11,1.571,0.524
7,mm.MDN.006,20210304,706877.0,funding_fundraise,45,2,1,2,1.000,0.044
8,mm.MDN.006,20210304,706877.0,online_engag,12,26,7,61,2.346,5.083
9,mm.MDN.006,20210304,706877.0,online_event,8,1,2,2,2.000,0.250


## Get museum sample to inspect results

- Inspect the summary file on the rows that have been assigned to you (evaluator column). These rows represent indicators for museums in a given session, only including matches that have been considered valid by the system.
- For each row in the summary file, you can find the corresponding matches in the matches file. Please check them for quality, comparing as usual the annotation tokens with the page tokens.
- I realise that there are a lot of rows, so you can select a subsample of your choice. This is an exploratory task, so it’s mainly to check if the results make sense and the accuracy seems to be about 80% (basically, max 20% false positives). If needed, we can do also it as a formal evaluation. 

In [189]:
musem_sample_ids = muse_indic_sent_df.muse_id.sample(10,random_state=10)
muse_sample_df = muse_indic_sent_df[muse_indic_sent_df.muse_id.isin(musem_sample_ids)]
muse_sample_df.to_excel(out_folder+'tmp/museum_website_match_sample10_summary.xlsx',index=False)
allsess_match_valid_df[allsess_match_valid_df.muse_id.isin(musem_sample_ids)].to_excel(out_folder+'tmp/museum_website_match_sample10_matches.xlsx',index=False)

muse_sample_df

,muse_id,session_id,page_id,indicator_code,n_indic_all_ann_examples,n_uniq_sentences,n_matched_annotations,n_matches,matches_to_sent_ratio,matches_to_example_ratio
4261,mm.aim.0788,20210304,375954.0,closed_cur,21,2,9,11,5.500,0.524
4262,mm.aim.0788,20210304,375954.0,funding_fundraise,45,1,1,1,1.000,0.022
4263,mm.aim.0788,20210304,375954.0,online_engag,12,3,6,7,2.333,0.583
4264,mm.aim.0788,20210304,375954.0,open_cur,3,1,1,1,1.000,0.333
4265,mm.aim.0788,20210304,375954.0,reopen_intent,35,2,17,17,8.500,0.486
...,...,...,...,...,...,...,...,...,...,...
24384,mm.musa.242,20210404,271710.0,online_engag,12,3,2,3,1.000,0.250
24385,mm.musa.242,20210404,271710.0,open_cur,3,3,2,3,1.000,1.000
24386,mm.musa.242,20210404,271710.0,reopen_intent,35,1,1,1,1.000,0.029
24387,mm.musa.242,20210404,271710.0,reopen_plan,4,1,2,2,2.000,0.500


End of notebook